In [1]:
from constants import Split
from data.datastructures.hyperparameters.dpr import DenseHyperParams
from retriever.Contriever import Contriever
from data.loaders.RetrieverDataset import RetrieverDataset
from metrics.SimilarityMatch import CosineSimilarity as CosScore

In [2]:
config_instance = DenseHyperParams(query_encoder_path="facebook/contriever",
                                   document_encoder_path="facebook/contriever",
                                   batch_size=32)

loader = RetrieverDataset("wikimultihopqa", "wiki-musiqueqa-corpus", "evaluation/config.ini", Split.DEV)
queries, qrels, corpus = loader.qrels()

print("info: ", len(queries), len(qrels), len(corpus), queries[0], qrels)

c:\Users\alexm\miniforge3\envs\bcqa\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Transforming passage dataset: 100%|██████████| 563424/563424 [00:01<00:00, 310256.41it/s]
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
c:\Users\alexm\miniforge3\envs\bcqa\lib\site-packages\transformers\tokenization_utils_base.py:2377: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='long

Harley-Davidson Harley-Davidson
KeysView(<Section: Data-Path>)
12576


100%|██████████| 1200/1200 [03:17<00:00,  6.07it/s]


Dataset loaded of length 12000
info:  1200 1200 563424 <data.datastructures.question.Question object at 0x00000291D2327E50> {'8813f87c0bdd11eba7f7acde48001122': {'314523': 1, '387778': 1, '550339': 1, '69975': 1, '126271': 1, '35800': 1, '369523': 1, '401226': 1, '237230': 1, '209617': 1}, '61a46987092f11ebbdaeac1f6bf848b6': {'546048': 1, '404896': 1, '458648': 1, '250316': 1, '40416': 1, '99061': 1, '143517': 1, '280200': 1, '380442': 1, '70651': 1}, 'e2a3bf2a0bdd11eba7f7acde48001122': {'271949': 1, '175303': 1, '79089': 1, '193912': 1, '214158': 1, '81390': 1, '273667': 1, '9415': 1, '438703': 1, '15376': 1}, '0cd3bdea0bde11eba7f7acde48001122': {'227835': 1, '322620': 1, '402608': 1, '73488': 1, '446247': 1, '103924': 1, '171055': 1, '354608': 1, '238514': 1, '107754': 1}, 'f9dcb4a60bda11eba7f7acde48001122': {'52714': 1, '141449': 1, '103781': 1, '73488': 1, '299688': 1, '419493': 1, '44408': 1, '171617': 1, '446247': 1, '322620': 1}, '8981e1ce0bb011ebab90acde48001122': {'219487': 1,

In [3]:
tasb_search = Contriever(config_instance)
similarity_measure = CosScore()
response = tasb_search.retrieve(corpus, queries, 100, similarity_measure)
# print(response)
print("indices",len(response))

c:\Users\alexm\miniforge3\envs\bcqa\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


token_emb torch.Size([1200, 35, 768])
sentence_emb torch.Size([1200, 768])
indices 1200


In [4]:
from metrics.retrieval.RetrievalMetrics import RetrievalMetrics

metrics = RetrievalMetrics(k_values=[1,10,100])
print(metrics.evaluate_retrieval(qrels=qrels, results=response))

({'NDCG@1': 0.445, 'NDCG@10': 0.21355, 'NDCG@100': 0.27421}, {'MAP@1': 0.04475, 'MAP@10': 0.10608, 'MAP@100': 0.12786}, {'Recall@1': 0.04475, 'Recall@10': 0.16096, 'Recall@100': 0.29363}, {'P@1': 0.445, 'P@10': 0.16008, 'P@100': 0.02923})


In [53]:
# Understand queries
print(queries[0].id())
print(queries[0].text())

# Qrels
qrel1 = next(iter(qrels.items()))
test_query_id, test_context_ids = qrel1

print(test_query_id)
print(test_context_ids)

# Corpus
print(corpus[401226].text())

8813f87c0bdd11eba7f7acde48001122
Who is the mother of the director of film Polish-Russian War (Film)?
8813f87c0bdd11eba7f7acde48001122
{'314523': 1, '387778': 1, '550339': 1, '69975': 1, '126271': 1, '35800': 1, '369523': 1, '401226': 1, '237230': 1, '209617': 1}
Xawery Żuławski (born 22 December 1971 in Warsaw) is a Polish film director. In 1995 he graduated National Film School in Łódź. He is the son of actress Małgorzata Braunek and director Andrzej Żuławski. His second feature "Wojna polsko-ruska" (2009), adapted from the controversial best-selling novel by Dorota Masłowska, won First Prize in the New Polish Films competition at the 9th Era New Horizons Film Festival in Wrocław. In 2013, he stated he intends to direct a Polish novel "Zły" by Leopold Tyrmand. Żuławski and his wife Maria Strzelecka had 2 children together: son Kaj Żuławski (born 2002) and daughter Jagna Żuławska (born 2009).


In [8]:
next(iter(response.items()))

You are given a question and you MUST respond by EXTRACTING the answer (max 5 tokens) from the provided document.
Document: Maheen Khan is a Pakistani fashion and costume designer, also an award winner fashion designer for fashion labels like" The Embroidery HouseMaheen" and" Gulabo". She has done many national and international fashion events and shows. She undertook embroidery for the film Snow White and the Huntsman and television series The Jewel in the Crown.
Question: Who is the mother of the director of film Polish-Russian War (Film)?
Answer:

('8813f87c0bdd11eba7f7acde48001122',
 {'454664': 0.5472020506858826,
  '126271': 0.547201931476593,
  '258738': 0.47650963068008423,
  '83041': 0.47294431924819946,
  '474600': 0.47294411063194275,
  '96516': 0.46978694200515747,
  '420403': 0.4680063724517822,
  '401699': 0.46553969383239746,
  '373732': 0.4630969762802124,
  '185071': 0.4591957926750183,
  '269921': 0.4591957628726959,
  '8011': 0.4586009681224823,
  '99985': 0.45826271176338196,
  '76308': 0.45452091097831726,
  '507138': 0.45342355966567993,
  '401226': 0.45342352986335754,
  '2526': 0.4518086016178131,
  '28189': 0.45150187611579895,
  '109697': 0.4514307379722595,
  '409901': 0.451043963432312,
  '235356': 0.4497726559638977,
  '134031': 0.44857826828956604,
  '291328': 0.44778573513031006,
  '344210': 0.4472500681877136,
  '42697': 0.44610223174095154,
  '468054': 0.44610217213630676,
  '431539': 0.4452982246875763,
  '193464': 0.4452981948852539,
  '462545': 0.4452981948852539,
  '563068': 0.4447028636932373,
 

In [44]:
import numpy as np

# strategy = "top_k_plus_hard_negative_docs"
strategy = "top_k_plus_hard_negative_docs"
n_additional_docs = 10
top_k = 5

q_contexts = {}

for q_id, contexts in response.items():
    context_ids = list(map(int, contexts.keys()))
    q_contexts[q_id] = context_ids[:top_k]
    if strategy == "top_k_plus_random_docs":
        # Sample n_additional_docs random docs. If one of them is already in the top k, sample again
        while len(q_contexts[q_id]) < top_k + n_additional_docs:
            random_doc = np.random.choice(len(corpus), 1)[0]
            if random_doc not in context_ids:
                q_contexts[q_id].append(random_doc)
                
    elif strategy == "top_k_plus_hard_negative_docs":
        # Sample n_additional_docs hard negative docs from context_ids. A hard negative doc is a doc that is relevant to another query
        # but not to the current query and it is not present in the current query's ground truth
        gt_contexts = list(map(int, qrels[q_id].keys()))
        # Make a new list of context_ids that are not in the current query's ground truth. Use filter
        context_ids = list(filter(lambda x: x not in gt_contexts, context_ids))
        # Concat n_additional_docs hard negative docs
        q_contexts[q_id] += context_ids[:n_additional_docs]

In [45]:
print(q_contexts)
# Save q_contexts to a file
np.save("{}_{}_{}.npy".format(strategy, top_k, n_additional_docs), q_contexts)

{'8813f87c0bdd11eba7f7acde48001122': [454664, 126271, 258738, 83041, 474600, 454664, 258738, 83041, 474600, 96516, 420403, 401699, 373732, 185071, 269921], '61a46987092f11ebbdaeac1f6bf848b6': [458648, 143517, 250316, 404896, 70651, 136173, 404594, 93064, 486842, 301988, 4173, 142315, 561133, 452718, 120109], 'e2a3bf2a0bdd11eba7f7acde48001122': [193912, 346210, 81390, 273667, 409987, 346210, 409987, 246891, 491634, 290377, 374186, 431980, 140941, 7254, 529137], '0cd3bdea0bde11eba7f7acde48001122': [196475, 548430, 337684, 164155, 242223, 196475, 548430, 337684, 164155, 242223, 430896, 163305, 7066, 152580, 133073], 'f9dcb4a60bda11eba7f7acde48001122': [171617, 230191, 59045, 167958, 229646, 230191, 59045, 167958, 229646, 72426, 313266, 407251, 526265, 106999, 108623], '8981e1ce0bb011ebab90acde48001122': [150999, 530007, 259546, 482753, 493202, 530007, 493202, 140824, 259191, 473721, 557323, 369800, 26574, 168919, 85102], '019bed300bde11eba7f7acde48001122': [135801, 417200, 470046, 178480,